# Pokedex Generator
Takes the following information regarding the pokemons
- National Pokedex ID (ndex)
- Regional Pokedex ID (rdex)
- Pokemon's Name
- Type/s
- URL page to the Pokemon

### Import Packages

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import Counter

### Get and parse URL information

In [2]:
url = 'https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number'
base_url = 'bulbapedia.bulbagarden.net'
main_page = requests.get(url)
main_soup = BeautifulSoup(main_page.content, 'html.parser')

### Locate elements that contain the necessary information

In [3]:
main_page_content = main_soup.find('div', id = 'mw-content-text')
main_page_tables = main_page_content.find_all('table')

### Create Scraper Function

In [4]:
def scrape(dex_content, dex_index):
    indexed_table = zip(dex_content, dex_index)
    poke_info_list = []

    for poke_val, index_val in indexed_table:
        #skips the odd indexed values on the table since it is blanked
        if index_val%2 == 0 and index_val != 0:
            raw_poke_info = poke_val.text.strip().split('\n')
            raw_links = poke_val.find('a', href = True)
            poke_url = base_url + str(raw_links['href'])

            # Normal form, double type
            if len(raw_poke_info) == 10:
                rdex = raw_poke_info[0]
                ndex = raw_poke_info[2]
                name = raw_poke_info[6]
                type_1 = raw_poke_info[8]
                type_2 = raw_poke_info[9]
                cat = 'Normal Form Double Type'

            # Normal form, single type
            elif len(raw_poke_info) == 9:
                rdex = raw_poke_info[0]
                ndex = raw_poke_info[2]
                name = raw_poke_info[6]
                type_1 = raw_poke_info[8]
                type_2 = ''
                cat = 'Normal Form Single Type'

            # Regional form, double type
            elif len(raw_poke_info) == 8:
                rdex = ''
                ndex = raw_poke_info[0]
                name = raw_poke_info[4]
                type_1 = raw_poke_info[6]
                type_2 = raw_poke_info[7]
                cat = 'Regional Form Double Type'

            # Regional form, single type
            elif len(raw_poke_info) == 7:
                rdex = ''
                ndex = raw_poke_info[0]
                name = raw_poke_info[4]
                type_1 = raw_poke_info[6]
                type_2 = ''
                cat = 'Regional Form Single Type'

            else:
                print('Check row with length of', len(raw_poke_info))

            poke_info_list.append((rdex, ndex, name, type_1, type_2, cat, poke_url))
            
            labels = ['rdex', 'ndex', 'name', 'type_1', 'type_2', 'category', 'pokemon_wiki_url']
            df_poke_info_list = pd.DataFrame(poke_info_list, columns = labels)
            
            df_poke_info_list.to_csv('Pokemon Generation ' + str(y) + ' List CSV.csv', index = False)
            df_poke_info_list.to_json('Pokemon Generation ' + str(y) + ' List JSON.json', orient = 'records')

### Execute

In [5]:
poke_gen_index = range(0,9)
full_poke_table = zip(main_page_tables, poke_gen_index)
generation = 1

for x, y in full_poke_table:
    dex_content = x.contents[1]
    dex_index = range(0, len(dex_content))
    scrape(dex_content, dex_index)
    print('Exported Gen', generation, 'to CSV and JSON')
    generation += 1

Exported Gen 1 to CSV and JSON
Exported Gen 2 to CSV and JSON
Exported Gen 3 to CSV and JSON
Exported Gen 4 to CSV and JSON
Exported Gen 5 to CSV and JSON
Exported Gen 6 to CSV and JSON
Exported Gen 7 to CSV and JSON
Exported Gen 8 to CSV and JSON
Exported Gen 9 to CSV and JSON
